In [1]:
import requests
from bs4 import BeautifulSoup
import unicodedata
import pandas as pd
import re
import time

iyakuinfo　速すぎる

In [2]:
data=pd.read_csv("../merged/len3_ndb_blp_DN_firm.csv",encoding="utf-8",index_col=0)
data.head()


C:\Users\hakos\AppData\Local\Temp\ipykernel_19252\1102737598.py:1: DtypeWarning: Columns (118) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("../merged/len3_ndb_blp_DN_firm.csv",encoding="utf-8",index_col=0)


,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価_x,後発品区分,総計,year,in_hospital,...,先発医薬品,同一剤形・規格の後発医薬品がある先発医薬品,薬価_y,経過措置による使用期限,備考,a,b,c,d,_merge
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,先発品,○,13.1,NaN,NaN,NaN,NaN,NaN,NaN,both
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,先発品,○,23.2,NaN,NaN,NaN,NaN,NaN,NaN,both
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,先発品,○,13.1,NaN,NaN,NaN,NaN,NaN,NaN,both
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,先発品,○,23.2,NaN,NaN,NaN,NaN,NaN,NaN,both
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,先発品,○,63.1,NaN,NaN,NaN,NaN,NaN,NaN,both


In [3]:
# set(data["メーカー名"])

In [4]:
# test
# ここにあなたのHTMLコードを設定してください
url="https://www.iyaku.info/medicine/?sk2=ジスチグミン臭化物錠５ｍｇ「テバ」&sk3=&sf=1#st"

r=requests.get(url)
soup=BeautifulSoup(r.content,"html.parser")
# 指定されたCSSセレクターを使用して要素を取得します
target_element = soup.select_one("#medicine > div.form_search2.mt40 > table.tb_bot > tr:nth-child(1) > td.w210.pl00 > p")
name=soup.select_one("#medicine > div.form_search2.mt40 > table:nth-child(3) > tr:nth-child(1) > td.w500 > p > strong")
# 要素が存在するか確認します
if target_element:
    # 要素のテキストを取得します
    text = target_element.get_text()
    name=name.get_text()
    print(name,text)
else:
    print("要素が見つかりませんでした")


ジスチグミン臭化物錠５ｍｇ「テバ」 武田テバ薬品


In [5]:
med_names=list(set(data.loc[data["メーカー名"].isna(),"医薬品名"]))
len(med_names)

52

cleanしても意味がなかった

In [6]:
name2cleaned = {}
cleaned=[]
for med_name in med_names:
    # クリーン処理を実行
    cleaned_name = unicodedata.normalize('NFKC', re.sub(r'\u3000', ' ', re.search(r'.*\u3000', med_name).group()) if re.search(r'.*\u3000', med_name) else med_name)
    
    cleaned.append(cleaned_name)
    # クリーン前のテキストをキー、クリーン後のテキストを値として辞書に追加
    name2cleaned[med_name] = cleaned_name
len(cleaned)

52

In [7]:
cleaned[:5]

['アンブロキソール塩酸塩内用液0.75%「JG」',
 'ビスコポール外用液1%',
 'マーヨン錠「50μg」 ',
 'アンブロキソール塩酸塩内用液0.75%「ツルハラ」',
 'アンブロキソール塩酸塩内用液0.75%「タイヨー」']

In [8]:
# matched=[]
# for name in cleaned:
#     pattern = r'「(.*?)」'
#     matches = re.findall(pattern, name,flags=re.MULTILINE)
#     for match in matches:
#         # print(match)
#         matched.append(match)
# # pd.Series(matched).value_counts(),len(matched)

In [9]:
# data_lack=data[data["メーカー名"].isna()]
name2firm={}
name2name={}
failed=[]
for i,med_name in enumerate(med_names):
    url=f"https://www.iyaku.info/medicine/?sk2={med_name}&sk3=&sf=1#st"
    # print(url)
    r=requests.get(url)
    soup=BeautifulSoup(r.content,"html.parser")
    # 指定されたCSSセレクターを使用して要素を取得します
    target_element = soup.select_one("#medicine > div.form_search2.mt40 > table.tb_bot > tr:nth-child(1) > td.w210.pl00 > p")
    name=soup.select_one("#medicine > div.form_search2.mt40 > table:nth-child(3) > tr:nth-child(1) > td.w500 > p > strong")
    # 要素が存在するか確認します
    if target_element:
        # 要素のテキストを取得します
        text = target_element.get_text()
        name=name.get_text()
        name2name[med_name]=name
        name2firm[med_name]=text
        # print(text)
    else:
        failed.append(med_name)
        # print("要素が見つかりませんでした")
    print(i, end=" ")
    time.sleep(1)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 

In [10]:
len(name2firm)

25

In [16]:
import pickle

file_path = './pickle/name2firm_ii.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(name2firm, file)
file_path = './pickle/name2name_ii.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(name2name, file)

In [12]:
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm)

In [13]:
data_lack=data[data["メーカー名"].isna()]
data_lack["医薬品名"].value_counts(),data_lack["薬価基準収載医薬品コード"].value_counts()

(ピロキシカム坐剤２０ｍｇ「ＪＧ」                2
 アポラキート錠２００ｍｇ                    2
 アシクロビル錠２００ｍｇ「ファイザー」             2
 ビクロックス錠２００　２００ｍｇ                2
 ビフォノールクリーム１％                    2
 ビスコポールクリーム１％                    2
 マリンゾールクリーム１％                    2
 ビスコポール外用液１％                     2
 マイコゾールクリーム１％                    2
 ビルミチンクリーム１％                     2
 ビクロノール外用液１％                     2
 ビクロノールクリーム１％                    2
 ラニチジン錠１５０ｍｇ「サワイ」                2
 ラニチジン錠１５０ｍｇ「日医工」                2
 ロペラミド塩酸塩細粒０．１％「フソー」             2
 ロペラミド塩酸塩細粒小児用０．０５％「タイヨー」        2
 ツロブテロール塩酸塩１ｍｇ錠                  2
 マーヨン錠「５０μｇ」　０．０５ｍｇ              2
 エプカロール錠５０μｇ　０．０５ｍｇ              2
 プロカテロール塩酸塩錠５０μｇ「日医工」　０．０５ｍｇ     2
 プロカテロール塩酸塩錠５０μｇ「サワイ」　０．０５ｍｇ     2
 エプカロール錠２５μｇ　０．０２５ｍｇ             2
 塩酸アンブロキソール内用液０．７５％「ＰＨ」          2
 アシクロビル錠２００ｍｇ「テバ」                2
 チモロール点眼液Ｔ０．５％                   1
 フラボキサート塩酸塩錠２００ｍｇ「フソー」           1
 プロカテロール塩酸塩錠２５μｇ「トーワ」　０．０２５ｍｇ    1
 Name: 医薬品名, dtype: int64,
 2655708N1016    12
 2259004

In [14]:
data.shape
# data.to_csv("../merged/len3_ndb_blp_DN_firm_filled.csv",encoding="utf-8")

(25798, 131)

In [15]:
name2name

{'アンブロキソール塩酸塩内用液０．７５％「ＪＧ」': 'アンブロキソール塩酸塩内用液０．７５％「ＪＧ」',
 'アンブロキソール塩酸塩内用液０．７５％「ツルハラ」': 'アンブロキソール塩酸塩内用液０．７５％「ツルハラ」',
 'アンブロキソール塩酸塩内用液０．７５％「タイヨー」': 'アンブロキソール塩酸塩内用液０．７５％「タイヨー」',
 'ツロブテロール塩酸塩ＤＳ小児用０．１％「タカタ」': 'ツロブテロール塩酸塩ＤＳ小児用０．１％「タカタ」',
 'オキシブチニン塩酸塩錠２ｍｇ「トーワ」': 'オキシブチニン塩酸塩錠２ｍｇ「トーワ」',
 'ビホナゾールクリーム１％「ＹＤ」': 'ビホナゾールクリーム１％「ＹＤ」',
 'クロトリマゾールクリーム１％「イワキ」': 'クロトリマゾールクリーム１％「イワキ」',
 'アシクロビル錠２００ｍｇ「トーワ」': 'アシクロビル錠２００ｍｇ「トーワ」',
 'ツロブテロール塩酸塩錠１ｍｇ「トーワ」': 'ツロブテロール塩酸塩錠１ｍｇ「トーワ」',
 'フラボキサート塩酸塩錠２００ｍｇ「日医工」': 'フラボキサート塩酸塩錠２００ｍｇ「日医工」',
 'アシクロビル錠２００ｍｇ「サワイ」': 'アシクロビル錠２００ｍｇ「サワイ」',
 'ツロブテロール塩酸塩ＤＳ０．１％「オーハラ」': 'ツロブテロール塩酸塩ＤＳ０．１％「オーハラ」',
 'イルソグラジンマレイン酸塩錠２ｍｇ「日医工」': 'イルソグラジンマレイン酸塩錠２ｍｇ「日医工」',
 'トリメブチンマレイン酸塩錠１００ｍｇ「トーワ」': 'トリメブチンマレイン酸塩錠１００ｍｇ「トーワ」',
 'ツロブテロール塩酸塩ＤＳ小児用０．１％「トーワ」': 'ツロブテロール塩酸塩ＤＳ小児用０．１％「トーワ」',
 'オキシブチニン塩酸塩錠２ｍｇ「日医工」': 'オキシブチニン塩酸塩錠２ｍｇ「日医工」',
 'エルサメット配合錠': 'エルサメット配合錠',
 'フラボキサート塩酸塩錠２００ｍｇ「サワイ」': 'フラボキサート塩酸塩錠２００ｍｇ「サワイ」',
 'トラニラストカプセル１００ｍｇ「ＣＨ」': 'トラニラストカプセル１００ｍｇ「ＣＨ」',
 'アシクロビル錠２００ｍｇ「ＣＨ」': 'アシクロビル錠２００ｍｇ「ＣＨ」',
 'トリメブチンマレイン酸塩細粒２０％「